In [22]:
%%time
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train / 255.0
X_test = X_test / 255.0
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test, y_test)

Epoch 1/10
1875/1875 [==============================] - 8s 3ms/step - loss: 0.1264 - accuracy: 0.9620
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0416 - accuracy: 0.9871
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0286 - accuracy: 0.9905
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0206 - accuracy: 0.9936
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0151 - accuracy: 0.9954
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0127 - accuracy: 0.9957
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0093 - accuracy: 0.9968
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0088 - accuracy: 0.9970
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0065 - accuracy: 0.9979
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0.0337 

[0.033699363470077515, 0.9927999973297119]

In [23]:
%%time
import keras
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

def get_dataset():
    batch_size = 64
    num_val_samples = 10000
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    X_val = X_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    X_train = X_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size),
    )

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = get_compiled_model()

train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)
model.evaluate(test_dataset)

Number of devices: 2
Epoch 1/10
782/782 [==============================] - 10s 9ms/step - loss: 0.1764 - accuracy: 0.9481 - val_loss: 0.0680 - val_accuracy: 0.9790
Epoch 2/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0512 - accuracy: 0.9841 - val_loss: 0.0533 - val_accuracy: 0.9832
Epoch 3/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0342 - accuracy: 0.9895 - val_loss: 0.0487 - val_accuracy: 0.9863
Epoch 4/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0242 - accuracy: 0.9926 - val_loss: 0.0511 - val_accuracy: 0.9867
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0173 - accuracy: 0.9949 - val_loss: 0.0450 - val_accuracy: 0.9888
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0135 - accuracy: 0.9957 - val_loss: 0.0523 - val_accuracy: 0.9873
Epoch 7/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0111 - accuracy: 0.9964 - val_loss: 0.0495 - val_a

[0.032730583101511, 0.9914000034332275]

In [24]:
%%time
import keras
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

def get_dataset():
    batch_size = 32
    num_val_samples = 10000
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    X_val = X_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    X_train = X_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size),
    )

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = get_compiled_model()

train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)
model.evaluate(test_dataset)

Number of devices: 2
Epoch 1/10
1563/1563 [==============================] - 15s 8ms/step - loss: 0.1440 - accuracy: 0.9571 - val_loss: 0.0584 - val_accuracy: 0.9833
Epoch 2/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0492 - accuracy: 0.9848 - val_loss: 0.0470 - val_accuracy: 0.9862
Epoch 3/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0313 - accuracy: 0.9904 - val_loss: 0.0510 - val_accuracy: 0.9865
Epoch 4/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0207 - accuracy: 0.9938 - val_loss: 0.0583 - val_accuracy: 0.9858
Epoch 5/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0151 - accuracy: 0.9952 - val_loss: 0.0547 - val_accuracy: 0.9868
Epoch 6/10
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0128 - accuracy: 0.9955 - val_loss: 0.0535 - val_accuracy: 0.9895
Epoch 7/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0100 - accuracy: 0.9967 - val_

[0.03677351027727127, 0.991100013256073]

In [25]:
%%time
import keras
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

def get_dataset():
    batch_size = 128
    num_val_samples = 10000
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    X_val = X_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    X_train = X_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size),
    )

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = get_compiled_model()

train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)
model.evaluate(test_dataset)

Number of devices: 2
Epoch 1/10
391/391 [==============================] - 7s 11ms/step - loss: 0.2470 - accuracy: 0.9290 - val_loss: 0.0770 - val_accuracy: 0.9784
Epoch 2/10
391/391 [==============================] - 4s 9ms/step - loss: 0.0689 - accuracy: 0.9789 - val_loss: 0.0575 - val_accuracy: 0.9837
Epoch 3/10
391/391 [==============================] - 4s 9ms/step - loss: 0.0477 - accuracy: 0.9857 - val_loss: 0.0496 - val_accuracy: 0.9852
Epoch 4/10
391/391 [==============================] - 4s 9ms/step - loss: 0.0368 - accuracy: 0.9890 - val_loss: 0.0514 - val_accuracy: 0.9848
Epoch 5/10
391/391 [==============================] - 4s 10ms/step - loss: 0.0286 - accuracy: 0.9914 - val_loss: 0.0549 - val_accuracy: 0.9843
Epoch 6/10
391/391 [==============================] - 4s 9ms/step - loss: 0.0221 - accuracy: 0.9933 - val_loss: 0.0453 - val_accuracy: 0.9880
Epoch 7/10
391/391 [==============================] - 4s 9ms/step - loss: 0.0173 - accuracy: 0.9947 - val_loss: 0.0435 - val_

[0.04500793293118477, 0.9868000149726868]

In [26]:
%%time
import keras
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

def get_dataset():
    batch_size = 512
    num_val_samples = 10000
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    X_val = X_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    X_train = X_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size),
    )

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = get_compiled_model()

train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)
model.evaluate(test_dataset)

Number of devices: 2
Epoch 1/10
98/98 [==============================] - 5s 22ms/step - loss: 0.4708 - accuracy: 0.8715 - val_loss: 0.1300 - val_accuracy: 0.9634
Epoch 2/10
98/98 [==============================] - 1s 15ms/step - loss: 0.1141 - accuracy: 0.9649 - val_loss: 0.0797 - val_accuracy: 0.9769
Epoch 3/10
98/98 [==============================] - 2s 16ms/step - loss: 0.0778 - accuracy: 0.9762 - val_loss: 0.0662 - val_accuracy: 0.9810
Epoch 4/10
98/98 [==============================] - 2s 16ms/step - loss: 0.0607 - accuracy: 0.9816 - val_loss: 0.0636 - val_accuracy: 0.9816
Epoch 5/10
98/98 [==============================] - 2s 16ms/step - loss: 0.0516 - accuracy: 0.9842 - val_loss: 0.0593 - val_accuracy: 0.9831
Epoch 6/10
98/98 [==============================] - 2s 16ms/step - loss: 0.0441 - accuracy: 0.9866 - val_loss: 0.0564 - val_accuracy: 0.9843
Epoch 7/10
98/98 [==============================] - 2s 16ms/step - loss: 0.0381 - accuracy: 0.9883 - val_loss: 0.0548 - val_accuracy:

[0.036035552620887756, 0.9900000095367432]

In [27]:
%%time
import keras
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

def get_dataset():
    batch_size = 1024
    num_val_samples = 10000
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    X_val = X_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    X_train = X_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size),
    )

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = get_compiled_model()

train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)
model.evaluate(test_dataset)

Number of devices: 2
Epoch 1/10
49/49 [==============================] - 5s 37ms/step - loss: 0.7184 - accuracy: 0.8138 - val_loss: 0.2230 - val_accuracy: 0.9361
Epoch 2/10
49/49 [==============================] - 1s 24ms/step - loss: 0.1809 - accuracy: 0.9460 - val_loss: 0.1198 - val_accuracy: 0.9684
Epoch 3/10
49/49 [==============================] - 1s 24ms/step - loss: 0.1106 - accuracy: 0.9668 - val_loss: 0.0866 - val_accuracy: 0.9760
Epoch 4/10
49/49 [==============================] - 1s 24ms/step - loss: 0.0806 - accuracy: 0.9757 - val_loss: 0.0702 - val_accuracy: 0.9802
Epoch 5/10
49/49 [==============================] - 1s 24ms/step - loss: 0.0633 - accuracy: 0.9811 - val_loss: 0.0600 - val_accuracy: 0.9837
Epoch 6/10
49/49 [==============================] - 1s 25ms/step - loss: 0.0528 - accuracy: 0.9839 - val_loss: 0.0543 - val_accuracy: 0.9847
Epoch 7/10
49/49 [==============================] - 1s 25ms/step - loss: 0.0457 - accuracy: 0.9861 - val_loss: 0.0516 - val_accuracy:

[0.04029170423746109, 0.986299991607666]

In [28]:
%%time
import keras
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

def get_dataset():
    batch_size = 256
    num_val_samples = 10000
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    X_val = X_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    X_train = X_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size),
    )

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = get_compiled_model()

train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=10, validation_data=val_dataset)
model.evaluate(test_dataset)

Number of devices: 2
Epoch 1/10
196/196 [==============================] - 6s 17ms/step - loss: 0.3709 - accuracy: 0.9004 - val_loss: 0.0932 - val_accuracy: 0.9756
Epoch 2/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0847 - accuracy: 0.9742 - val_loss: 0.0651 - val_accuracy: 0.9813
Epoch 3/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0575 - accuracy: 0.9826 - val_loss: 0.0542 - val_accuracy: 0.9838
Epoch 4/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0448 - accuracy: 0.9864 - val_loss: 0.0481 - val_accuracy: 0.9871
Epoch 5/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0363 - accuracy: 0.9891 - val_loss: 0.0459 - val_accuracy: 0.9875
Epoch 6/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0293 - accuracy: 0.9912 - val_loss: 0.0456 - val_accuracy: 0.9874
Epoch 7/10
196/196 [==============================] - 2s 12ms/step - loss: 0.0243 - accuracy: 0.9930 - val_loss: 0.0476 -

[0.0314968079328537, 0.989799976348877]

In [11]:
%%time
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
x_train, x_test = x_train / 255.0, x_test / 255.0
model.fit(x_train, y_train, epochs=10)
model.evaluate(x_test, y_test)


Epoch 1/10
1875/1875 [==============================] - 13s 6ms/step - loss: 0.1260 - accuracy: 0.9612
Epoch 2/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0413 - accuracy: 0.9868
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0281 - accuracy: 0.9913
Epoch 4/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0203 - accuracy: 0.9935
Epoch 5/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0161 - accuracy: 0.9945
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0122 - accuracy: 0.9959
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0096 - accuracy: 0.9967
Epoch 8/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0079 - accuracy: 0.9972
Epoch 9/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0070 - accuracy: 0.9976
Epoch 10/10
313/313 [==============================] - 2s 5ms/step - loss

[0.03963141515851021, 0.9902999997138977]

In [14]:
%%time
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10)
    model.evaluate(X_test, y_test)

Epoch 1/10
1875/1875 [==============================] - 12s 5ms/step - loss: 0.1334 - accuracy: 0.9599
Epoch 2/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0433 - accuracy: 0.9864
Epoch 3/10
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0298 - accuracy: 0.9909
Epoch 4/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0211 - accuracy: 0.9935
Epoch 5/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0147 - accuracy: 0.9955
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0126 - accuracy: 0.9958
Epoch 7/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0091 - accuracy: 0.9969
Epoch 8/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0084 - accuracy: 0.9974
Epoch 9/10
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0060 - accuracy: 0.9980
Epoch 10/10
313/313 [==============================] - 2s 5ms/step - loss